# タスク 2: Amazon Bedrock ナレッジベースと Retrieve API を使用して質問応答アプリケーションを構築する

このタスクでは、Amazon Bedrock ナレッジベースと **Retrieve API** を使用して質問応答アプリケーションを構築します。ここでは、ナレッジベースにクエリを実行して、類似検索に基づいて必要な数のドキュメントチャンクを取得します。次に、関連ドキュメントでプロンプトを強化し、Amazon Nova Lite への入力として機能するクエリを実行して応答を生成します。

ナレッジベースを使用すると、Amazon Bedrock の基盤モデル (FM) を会社のデータに安全に接続して、検索拡張生成 (RAG) を行うことができます。追加データにアクセスすることで、FM を継続的に再トレーニングしなくても、モデルがコンテキストに応じた関連性の高い正確な応答を生成できるようになります。ナレッジベースから取得された情報にはすべて出典の属性が付き、透明性を高めると共にハルシネーションを最低限に抑えます。

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i>**詳細情報:** コンソールを使用してナレッジベースを作成する方法の詳細については、**[Amazon Bedrock ナレッジベース](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html)** を参照してください。

### シナリオ

検索拡張生成 (RAG) パターンを使用してソリューションを実装します。RAG は、言語モデルの外部からデータを取得し、取得データをコンテキストに追加することでプロンプトを拡張します。ここでは、ラボのプロビジョニングの一環として作成されたナレッジベースで RAG を効果的に実行します。
    
このノートブックで行うこと:

- AnyCompany の 10K 財務報告 (合成的に生成されたデータセット) をテキストコーパスとして使用して質問応答を行います。このデータは、ラボのプロビジョニング中に既に Amazon Bedrock ナレッジベースに取り込まれています。
- このラボ環境用に作成された既存のナレッジベースのナレッジベース ID を使用します。
- Amazon Bedrock **Retrieve API** と LangChain 検索の両方を使用して、ナレッジベースからドキュメントを取得し、それをコンテキストとして追加して、ユーザーのクエリに回答します。

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **注意:** [**Run**] メニューの [**Run All Cells**] オプションを使用するよりも、各コードセルを個別に実行することをお勧めします。すべてのセルをまとめて実行すると、カーネルがクラッシュしたり再起動したりするなど、予期しない動作が発生することがあります。セルを 1 つずつ実行することで、実行フローをより適切に制御し、潜在的なエラーを早期に検出し、コードを意図したとおりに実行することができます。

### タスク 2.1: 環境を設定する

このタスクでは、Amazon Bedrock クライアントを起動し、次の操作を実行します。

- ナレッジベース ID を確認します。
- 必要なライブラリをインポートし、必要なクライアントを設定します。

#### タスク 2.1.1: ナレッジベース ID を確認する

このノートブックを実行するには、ナレッジベース ID を確認して **kb_id** 変数に割り当てて、必要なパッケージをインストールする必要があります。

1. 次のコードセルを実行して、Amazon Bedrock の既存のナレッジベースの ID を確認します。

In [1]:
import boto3
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

#### タスク 2.1.2: Amazon Bedrock クライアントを起動する

2. 次のコードセルを実行して、環境を設定するために必要なライブラリをインポートします。

In [ ]:
import boto3
from botocore.client import Config
import pprint
import json

pp = pprint.PrettyPrinter(indent=2)

session = boto3.session.Session()
region = session.region_name

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)

## パート 1: Amazon Bedrock の基盤モデルで **Retrieve** API を使用する

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** このパートでは *amazon.nova-lite-v1:0* モデルを使用します。

### タスク 2.2: Amazon Bedrock の基盤モデルで Retrieve API を使用する

このタスクでは、Amazon Bedrock のナレッジベースによって提供される **Retrieve** API を呼び出す取得関数を定義します。こうすることにより、ユーザーのクエリが埋め込みに変換され、ナレッジベースが検索され、関連する結果が返されます。これにより、セマンティック検索結果に基づいてカスタムワークフローの構築をより細かく制御できるようになります。

**Retrieve** API の出力には、**取得したテキストチャンク**、ソースデータの**ロケーションタイプ**と **URI**、検索の関連性**スコア**が含まれます。**retrievalConfiguration** の **overrideSearchType** オプションを使用することもできます。このオプションでは、**HYBRID** または **SEMANTIC** のどちらを使用するかを選択できます。

デフォルトでは、最適な戦略が選択され、最も関連性の高い結果が得られます。ハイブリッド検索またはセマンティック検索を使用するようにデフォルトオプションをオーバーライドする場合は、値を **HYBRID/SEMANTIC** に設定できます。

<!-- ![retrieveAPI](./images/retrieveAPI.png) -->
<img src="images/retrieveAPI.png" width=50% height=20% />

**画像の説明: 上の図は、ラボ環境のカスタマイズされた RAG ワークフローを示しています。**

3. 次のコードセルを実行して、**Retrieve** API を呼び出す、**取得**関数を定義します:

In [3]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### タスク 2.2.1: 初期化された LLM からの応答のクエリを行う前にナレッジベース ID を初期化する

このタスクでは、**Retrieve** API を呼び出し、**ナレッジベース ID**、**結果の数**、**クエリ**をパラメータとして渡します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** 返された各テキストチャンクの関連スコアを表示できます。このスコアは、どの程度一致しているかという観点からクエリとの相関関係を示しています。

4. 次のコードセルを実行し、**ナレッジベース ID**、**結果の数**、**クエリ**をパラメータとして渡して、**Retrieve** API を呼び出します。

In [ ]:
query = "2022年12月31日現在のAnyCompanyのtotal operating lease liabilitiesとtotal sublease incomeはいくらでしたか？"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

#### タスク 2.2.2: **Retrieve** API の応答からテキストチャンクを抽出する

このタスクでは、**Retrieve** API の応答からテキストチャンクを抽出します。

5. 次の 2 つのコードセルを実行して、検索結果からコンテキストを取得して出力します。

In [5]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [ ]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

#### タスク 2.2.3: パーソナライズされた応答をモデルで生成するために特定のプロンプトを使用する 

このタスクでは、可能な限り事実に基づいた統計情報を使用して質問に回答する、ファイナンシャルアドバイザー AI システムとして機能するように、モデルに特定のプロンプトを使用します。ユーザー**クエリ**と共に、プロンプトの **{contexts}** の一部として前のタスクの **Retrieve** API 応答を指定して、モデルが参照できるようにします。

6. 次のコードセルを実行して、モデルがファイナンシャルアドバイザー AI システムとして機能するように、特定のプロンプトを使用します。

In [ ]:
prompt = f"""
Human: あなたはファイナンシャルアドバイザーのAIシステムであり、可能な限り事実に基づいた統計情報を用いて質問に答えます。
以下の情報を用いて、<question>タグで囲まれた質問に簡潔に回答してください。
わからない部分がある場合は、その部分は「わからない」とだけ伝え、わざわざ答えを作ろうとしないでください。
<context>
{contexts}
</context>

<question>
{query}
</question>

回答は具体的なものでなければならず、可能な場合は統計や数字を使用する必要があります。

Assistant:"""

#### タスク 2.2.4: Amazon Bedrock から基盤モデルを呼び出す

このタスクでは、Amazon Bedrock の **amazon.nova-lite-v1:0** 基盤モデルを使用します。

7. 次の 2 つのコードセルを実行して、Amazon Bedrock の **amazon.nova-lite-v1:0** 基盤モデルを呼び出します。コンテキストとクエリの両方をモデルに渡します。

In [ ]:
# payload with model parameters
messages = [{
    "role": "user",
    "content": [{"text": prompt}]
}]

# Create the proper Nova Lite payload
nova_payload = {
    "schemaVersion": "messages-v1",
    "messages": messages,
    "inferenceConfig": {
        "maxTokens": 512,
        "temperature": 0.5,
        "topP": 0.9,
        "topK": 20
    }
}

In [ ]:
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_client.invoke_model(
    body=json.dumps(nova_payload),
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

# Parse and extract the response
response_body = json.loads(response.get('body').read())

# Extract just the text from the response
response_text = ''
if 'output' in response_body and 'message' in response_body['output']:
    message_content = response_body['output']['message']['content']
    if message_content and isinstance(message_content, list):
        response_text = message_content[0].get('text', '')

# Print the response text
print(response_text)

## パート 2: LangChain 統合

### タスク 2.3: LangChain 統合

このタスクでは、LangChain の **AmazonKnowledgeBasesRetriever** クラスを使用して質問応答アプリケーションを構築します。ナレッジベースにクエリを実行して、類似検索に基づいて必要な数のドキュメントチャンクを取得します。その後、質問に回答するために、それを LangChain チェーンで統合し、ドキュメントチャンクとクエリを llm (**Amazon Nova Lite**) に渡します。

#### タスク 2.3.1: 環境の設定

このタスクでは、環境を設定します。

8. 次のコードセルを実行して、環境の設定に必要なパッケージをインポートします。

In [11]:
import langchain
from langchain_aws import ChatBedrock
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

#### タスク 2.3.2: Retrieve API を呼び出す AmazonKnowledgeBasesRetriever オブジェクトを作成する

このタスクでは、Amazon Bedrock ナレッジベースによって提供される **Retrieve** API を呼び出す、LangChain の **AmazonKnowledgeBasesRetriever** オブジェクトを作成します。これにより、ユーザーのクエリが埋め込みに変換され、ナレッジベースが検索され、関連する結果が返されるので、セマンティック検索結果に基づいてカスタムワークフローの構築をより細かく制御できるようになります。

9. 次のコードセルを実行して **AmazonKnowledgeBasesRetriever** オブジェクトを作成します。

In [ ]:
query = "2022年12月31日現在のAnyCompanyのtotal operating lease liabilitiesとtotal sublease incomeはいくらでしたか？"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.invoke(
        input=query
    )
for doc in docs:
    print(doc.page_content)
    print("------")

#### タスク 2.3.3: パーソナライズされた応答を得るために、モデルに特定のプロンプトを使用する

このタスクでは、可能な限り事実に基づいた統計情報を使用して質問に回答する、ファイナンシャルアドバイザー AI システムとして機能するように、モデルに特定のプロンプトを使用します。ユーザー**クエリ**と共に、プロンプトの **{context}** の一部として上記のタスクの **Retrieve** API 応答を指定して、モデルが参照できるようにします。

10.次のコードセルを実行して、モデルがファイナンシャルアドバイザー AI システムとして機能するように、特定のプロンプトを使用します。

In [ ]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: あなたはファイナンシャルアドバイザーのAIシステムであり、可能な限り事実に基づいた統計情報を用いて質問に答えます。
以下の情報を用いて、<question>タグで囲まれた質問に簡潔に回答してください。
わからない部分がある場合は、その部分は「わからない」とだけ伝え、わざわざ答えを作ろうとしないでください。
<context>
{context}
</context>

<question>
{question}
</question>

回答は具体的なものでなければならず、可能な場合は統計や数字を使用する必要があります。

Assistant:"""
nova_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

#### タスク 2.3.4: 取得チェーンでリトリーバーと LLM を統合して質問応答アプリケーションを構築する

このタスクでは、LangChain Expression Language (LCEL) を使用してリトリーバーと LLM を統合し、質問応答アプリケーションを構築します。

11.次のセルを実行し、retriever.invoke を使用して取得したドキュメントにリトリーバーと LLM を統合します。結果を出力します。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | nova_prompt
    | llm
    | StrOutputParser()
)

response=chain.invoke(query)
print(response)

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **タスクの完了:** このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じます。
- ラボセッションに戻り、タスク 3 に進みます。